In [1]:
from numpy.core.numeric import NaN
from MCtool.RFilter import gray
from genericpath import exists
from matplotlib import image
import math
import sys
import time

import cv2
from matplotlib import pyplot as plt
from tensorflow.python.keras.backend import dtype
from DeepLearning import LearnAndTest
from Rpkg.Rfund.InputFeature import InputFeature
import datetime
import os
import gc
import tensorflow as tf
import random
import numpy as np
import pandas as pd

from Rpkg.Rfund import ReadFile, WriteFile
from Rpkg.Rmodel import Unet, Mnet

import Filtering

import torch
from torch import nn


import DeepLearning
from tensorflow.keras.optimizers import Adam

from Rpkg.Rfund.InputFeature import InputFeature
from Rpkg.Rfund import ReadFile, WriteFile
from Rpkg.Rmodel import Unet, Mnet

from MCtool import RFilter, resultEval
from DeepLearning import save_eval_result

import numpy as np
import cv2
import torch
from transformations import ComposeDouble, FunctionWrapperDouble, create_dense_target, normalize_01
from customdatasets import SegmentationDataSet1
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
import pathlib
from skimage.transform import resize

#early stopping なし
from unet import UNet
from trainer import Trainer
from sklearn.model_selection import StratifiedKFold, train_test_split

import optuna
from optuna.visualization import plot_contour
from optuna.visualization import plot_edf
from optuna.visualization import plot_intermediate_values
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_rank
from optuna.visualization import plot_slice
from optuna.visualization import plot_timeline


import warnings

# Suppress specific PyTorch warnings
warnings.filterwarnings("ignore", message="Plan failed with a cudnnException")
warnings.filterwarnings("ignore", message="cudnn status: CUDNN_STATUS_NOT_SUPPORTED")



2024-12-25 19:04:00.751748: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-25 19:04:01.430898: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
def objective(trial):
    OUTPUT_DIR = '20241225-OptunaMCTransUnetTest'
    if trial.number % 10 == 0:
        print(f"Trial {trial.number}")


    HIDDEN_SIZE = trial.suggest_categorical('hidden_size', [512, 768, 1024, 1280])
    N_LAYER = trial.suggest_categorical('n_layer', [8, 10, 12, 14, 16])
    LR = trial.suggest_float('learning_rate', 1e-3, 1e-1)
    KERNEL_SIZE_CONV = trial.suggest_categorical('kernel_size_conv', [1, 3, 5, 7, 9])
    FUSION_OUT_CHANNEL = trial.suggest_categorical('post_fusion_out_channel', [1, 3, 5, 7, 9])
    DECODER_CHANNEL = trial.suggest_categorical('decoder_channel', [(512, 256, 128, 64), (256, 128, 64, 32), (256, 128, 64, 16), (128, 64, 32, 16)])
    PATCH_SIZE = trial.suggest_categorical('patch_size', [8, 16, 32])
    BATCH_SIZE = trial.suggest_categorical('batch_size', [2, 4, 6, 8, 10, 12, 14, 16])
    EPOCH = trial.suggest_int("epoch", 50, 300)

    # HIDDEN_SIZE = 768
    # N_LAYER = 12
    # LR = 0.01
    # KERNEL_SIZE_CONV = 1
    # FUSION_OUT_CHANNEL = 1
    # DECODER_CHANNEL = (256, 128, 64, 16)
    # PATCH_SIZE = 16
    # BATCH_SIZE = 2
    # EPOCH = 200


    IN_CHANNEL = 45

    AUGMENTED = False
    AUGMENTATION  =  30


    CROSS_VAL = False
    N_SPLIT = 4


    #FUSION_OUT_CHANNEL = trial.suggest_int('n_block', 1, 6)
        # Make sure there is no randomness in the output so that the output is reproduceable
    import torch
    import numpy as np
    import random

    # Set seed for Python random module
    random.seed(42)

    # Set seed for NumPy
    np.random.seed(42)

    # Set seed for PyTorch
    torch.manual_seed(42)

    # If you are using GPU
    torch.cuda.manual_seed(42)
    torch.cuda.manual_seed_all(42)

    # Make the convolution operations deterministic
    torch.backends.cudnn.deterministic = True

    # Disable the CUDNN benchmark to ensure deterministic results
    torch.backends.cudnn.benchmark = False

    # 自分の環境設定がうまくいったかどうかを確認しましょう、特にGPUの動作
    # Prints the version of PyTorch installed
    # print('PyTorch Version installed: ' + torch.__version__)

    # # Prints the version of CUDA associated with the installed PyTorch version
    # print('CUDA version associated with PyTorch version: ' + torch.version.cuda)

    # # Prints the version of cuDNN (CUDA Deep Neural Network library) being used by PyTorch
    # print('Version of cuDNN (CUDA Deep Neural Network library) being used by PyTorch' + str(torch.backends.cudnn.version()))

    # # Same as the line above
    # print('CUDA is available: ' + str(torch.cuda.is_available()))

    # # Returns the number of available CUDA-enabled GPUs
    # print('Number of GPUs compatible with CUDA:' + str(torch.cuda.device_count()))

    # # Returns the name of the GPU at index 0
    # print('Name of the GPU at index 0: '  + str(torch.cuda.get_device_name(0)))

    # # Returns the index of the current CUDA device being used
    # print('Current CUDA device index: '  + str(torch.cuda.current_device()))

    # # ファイル名の先頭部分（prefix）により自動的にファイル名を抽出するアルゴリズム。
    # # 実際それぞれのファイル名は違うと思うので、必須ではない

    """ 

    Extracts filenames in directory if they start with the prefix input 


    Args/Parameters:

        directory_path (string): The path of the dir (ex: /root/home/Documents/etc)
        
        prefix (string): Prefix of the file name (ex: 'Bo' is a prefix of 'Bone')

    Returns:

        sorted_file_names (list of str): File names sorted in ascending order in the dir without extension ex: ['bone1', 'bone2', ...]

    Raises:

        SomeError: ...

    """

    def file_names_with_prefix(directory_path, prefix):

        # Initialize an empty list to store the file names without extensions
        file_names_without_extension = []

        # Loop through all files in the specified directory
        for filename in os.listdir(directory_path):
            #Checking if the file in loop exists in the directory_path not sure how is this necessary
            #??
            if os.path.isfile(os.path.join(directory_path, filename)):
                # Check if the file name starts with the specified prefix
                if filename.startswith(prefix):
                    # Get the file name without extension
                    name_without_extension, _ = os.path.splitext(filename)

                    # Append the file name (without extension) to the list
                    file_names_without_extension.append(name_without_extension)

        # Sort the list of file names without extensions in ascending order
        sorted_file_names = sorted(
            file_names_without_extension,
            key=lambda x: (x.split('-')[0], int(x.split('-')[1]))
        )  # Modify this part based on your file naming convention

        # Now you have a sorted list of file names with the specified prefix and without extensions
        return sorted_file_names

    import pathlib
    from pathlib import Path

    # ここで、folder名とかPathとか色々設定

    # Setting the directory name, path and other settings

    # Define the root directory where your project is located
    # Defining a Path object for the project's root dir
    root_dir = Path(pathlib.Path.cwd())

    # result folder name
    #date_str = '20241202-Conv1x1-' + str(OUTPUT_DIR)
    date_str = OUTPUT_DIR

    # Define the directories for different types of data
    # Concatenating the root dir to the different dataset dirs
    data_dir = str(root_dir / "img_1006t/original")
    feature_dir = str(root_dir / "img_1006t/feature") 
    labeled_dir = str(root_dir / "img_1006t/labeled")

    augmented_labeled_dir = str(root_dir / "img_1006t/labelAug")
    augmented_data_dir = str(root_dir / "img_1006t/originalAug")
    augmented_feature_dir = str(root_dir / "img_1006t/featureAug")

    # annealing_img_dir = str(root_dir / "img_1006/annealing_img") # 焼きなまし法時に使う
    # annealing later, original for now
    annealing_img_dir = str(root_dir / "img_1006/original")
    result_dir = str(root_dir / "result" / date_str)
    test_result_dir= str(root_dir / "result_test" / date_str)

    # Making directories based on the path string result_dir and test_result_dir
    Path(result_dir).mkdir(parents=True, exist_ok=True)
    Path(test_result_dir).mkdir(parents=True, exist_ok=True)

    # Prints the paths of the dirs
    # print('Root directory: ' + str(root_dir))
    # print('Data directory (original dir): ' + str(data_dir))
    # print('Feature img directory: ' + str(feature_dir))
    # print('Labeled img directory: ' + str(labeled_dir))
    # print('Annealing directory: ' + str(annealing_img_dir))
    # print('Result directory: ' + str(result_dir))
    # print('Test result directory: ' + str(test_result_dir))

    # Defining variables filename list of path str starts with the prefix format
    # In this case: N1 and N3 is training data and N2 is validation data and N4 is a test data
    input_train = []
    input_name_val = []
    annealing_input_name = []
    input_train = []
    test_input_name = []


    # for raw_input_img in INPUT:
    #     input_train.extend(file_names_with_prefix(data_dir, raw_input_img))
    # for raw_val_img in VALIDATION:
    #     input_name_val.extend(file_names_with_prefix(data_dir, raw_val_img))
    # for raw_anneal in ANNEALING:
    #     annealing_input_name.extend(file_names_with_prefix(data_dir, raw_anneal))
    # for raw_test in TEST:
    #     test_input_name.extend(file_names_with_prefix(data_dir, raw_test))


    ####old version of assigning
    # input_train = file_names_with_prefix(data_dir, INPUT)
    # input_name_val = file_names_with_prefix(data_dir, VALIDATION)
    # annealing_input_name = file_names_with_prefix(data_dir, ANNEALING)
    # test_input_name = file_names_with_prefix(data_dir, TEST) 


    # extra_dataset = file_names_with_prefix(data_dir,'N5-')
    # input_train.extend(extra_dataset)

    # Prints the each data image name
    # print(input_train)
    # print(input_name_val)
    # print(annealing_input_name)
    # print(test_input_name)
    # print(extra_dataset)


    # Defining a var to store each list length
    len_train = len(input_train)
    len_val = len(input_name_val)
    len_test = len(test_input_name)
    len_annealing = len(annealing_input_name)


    # print(len(input_train))

    # print(len(input_name_val))
    # print(len(test_input_name))
    # print(len(annealing_input_name))

    # 特徴画像の特徴一覧をリストとして取得
    inputfeature_list = list(map(str, InputFeature))
    #print(inputfeature_list)

    feature_num = len(inputfeature_list)
    #print(feature_num)
    ## 重み計算なし
    def CreateWeightImage(input_number, augmentation=False):
        #print("Creating image arrays...")
        label_dataset = []
        arrDataset = []
        for i in input_number:
            if augmentation:
                label_path = os.path.join(augmented_labeled_dir, str(AUGMENTATION) + "aug/" , f"{i}.png")
            else:
                label_path = os.path.join(labeled_dir, f"{i}.png")
            input_originallabel = cv2.imread(label_path, cv2.IMREAD_GRAYSCALE)
            # commented the binary label because the project has more labels than 2 
            #_, binary_label = cv2.threshold(input_originallabel, 0, 255, cv2.THRESH_BINARY)
            
            ## This part was used for decreasing and increasing the label count when there was inconsistency with the label dataset
            
            # if len(np.unique(input_originallabel)) > 11:
            #     print("Defected image detected (more labels):" + f"{i}.png")
            #     print(np.unique(input_originallabel))
            #     for y in range(256):
            #         for x in range (256):
            #             if (input_originallabel[y][x] == 11) or (input_originallabel[y][x] == 12):
            #                 input_originallabel[y][x] = 0
            #     print(np.unique(input_originallabel))
            #     cv2.imwrite('testtttt.png', input_originallabel)
            # elif len(np.unique(input_originallabel)) < 11:
            #     print("Defected image detected (Less labels):" + f"{i}.png")
            #     defected = f"{i}_.png"
            #     print(np.unique(input_originallabel))
            #     path_to_mask = '/home/eric/Desktop/edit8label'

            #     # 25->9 32->10
            #     path_mask_abs = os.path.join(path_to_mask, defected)
            #     mask = cv2.imread(path_mask_abs, cv2.IMREAD_GRAYSCALE)
            #     for y in range(256):
            #         for x in range (256):
            #             if (input_originallabel[y][x] == 0) and (mask[y][x] == 25):
            #                 input_originallabel[y][x] = 9
            #             elif (input_originallabel[y][x] == 0) and (mask[y][x] == 32):
            #                 input_originallabel[y][x] = 10
            #     print(np.unique(input_originallabel))
            #     label_fixed_path = os.path.join(path_to_mask, f"{i}_fixed.png")
            #     cv2.imwrite(label_fixed_path, input_originallabel)
                        
            label_dataset.append(input_originallabel)

        #print("Number of label images:", len(label_dataset))

        for i in input_number:
            # changed this part from 100 to 256
            dataset_img = np.zeros((256, 256, feature_num), dtype=np.float32)

            for m in range(feature_num):
                if augmentation:
                    feature_img_path = os.path.join(augmented_feature_dir, str(AUGMENTATION) + "aug/" , str(i), f"{inputfeature_list[m]}.png")
                else:
                    feature_img_path = os.path.join(feature_dir, str(i), f"{inputfeature_list[m]}.png")
                input_featureimg = cv2.imread(feature_img_path, cv2.IMREAD_GRAYSCALE)
                dataset_img[:, :, m] = input_featureimg
            # after the loop the dataset_img size will be like: (256, 256, 108) and the tensor type is np array
            
            
            
            ##this part is dividing the dataset to use multiple conv1x1 operation
            #split_dataset_img = np.array_split(dataset_img, 4, axis = 2)
            #post_conv = np.zeros((256, 256, 4), dtype=np.float32)
            #concat_list = []
            #for index, part in enumerate(split_dataset_img):
                # after this operation shape will be from (256, 256, 27) to (1, 256, 256, 27)
                #part = torch.tensor(part, dtype = torch.float32).unsqueeze(0)
                # changed the shape for the conv again now it is (1, 27, 256, 256) (batch_size, channels, height, width)
                #part = part.permute(0, 3, 1, 2)
                #fusion = nn.Conv2d(in_channels = 27, out_channels = 1, kernel_size = 1, padding = 'same')
                #output_conv = fusion(part)
                #output_conv = output_conv.squeeze(0) #.permute(1, 2, 0) # now it is (256, 256, 1)
                #output_conv = output_conv.cpu().detach().numpy()
                #post_conv[:, :, index] = output_conv[0]
                #print((output_conv).shape)
                #concat_list.append(output_conv)
                
            #concat_output = torch.cat((concat_list[0], concat_list[1], concat_list[2], concat_list[3]), dim = 2)
            ## convert the concat result tensor into numpy array and it has to be on cpu to do the operation
            #print(np.unique(output_conv.detach()))
            #concat_output = concat_output.detach()
            #concat_output = concat_output.numpy()
            
            
            #arrDataset = concat_list
            arrDataset.append(dataset_img)

        arrDataset = np.array(arrDataset)
        # print("Completed creating image arrays:")
        # print("Dataset shape ", arrDataset.shape)
        # print("Label image shape ", np.shape(label_dataset))
        # print()

        return arrDataset, label_dataset

    def print_model_shapes(model, input_tensor):
        def forward_hook(module, input, output):
            1
            # print(f"Layer: {module.__class__.__name__}")
            # print(f"Input shape: {str(input[0].shape)}")
            # print(f"Output shape: {str(output.shape)}")
            # print("-----------------------")

        hooks = []
        for layer in model.children():
            hook = layer.register_forward_hook(forward_hook)
            hooks.append(hook)

        # print("Model Architecture:")
        # print(model)

        # Pass a dummy input tensor through the model to trigger the forward hooks
        with torch.no_grad():
            model(input_tensor)

        for hook in hooks:
            hook.remove()

    # preprocess function
    def preprocess(img: np.ndarray):
        img = np.moveaxis(img, -1, 0)  # Change from [H, W, C] to [C, H, W]
        img = normalize_01(img)  # Linear scaling to range [0-1]
        img = np.expand_dims(img, axis=0)  # Add batch dimension [B, C, H, W]
        img = img.astype(np.float32)  # Typecasting to float32
        #print("in pre")
        #print(np.unique(img))
        return img

    # postprocess function
    def postprocess(img: torch.tensor):
        img = torch.argmax(img, dim = 1)  # Perform argmax to generate 1 channel
        #img = img * 255.0 commented as the labels are from 0 to 11 in my case
        img = img.cpu().numpy().astype(np.uint8)  # Send to CPU and transform to numpy.ndarray
        # If batch_size > 1, you may need to loop through each batch and save them separately
        # If batch_size == 1, you can remove the batch dimension to save a single image

        # used for checking the unique label values whether if it is 0 to 11 or 0 to 255 scale
        #print("in post")
        #print(np.unique(img))


        img = np.squeeze(img)  # Remove batch dim and channel dim -> [H, W]
        # img = re_normalize(img)  # Scale it to the range [0-255]

        # If your image has multiple channels (C>1), like an RGB image, before saving with cv2.imwrite
        # you need to ensure the channel order is [B, G, R] instead of the common [R, G, B]
        # If C == 1, you can further reduce dimensions -> [H, W]
        if img.shape[0] == 3:  # [C, H, W]
            img = np.transpose(img, (1, 2, 0))  # [H, W, C]
            img = img[:, :, ::-1]  # Convert RGB to BGR
        elif img.shape[0] == 1:  # [C, H, W]
            img = np.squeeze(img, 0)  # [H, W]
        return img

    # earlystoppingあり
    # numpy形式のまま入力する用改良
    # 学習を行い予測結果画像を出力するとこまで
    from customdatasets import SegmentationDataSet0
    from customdatasets import SegmentationDataSet1
    from torch.utils.data import DataLoader

    def Learn_EA(input_dataset,label_dataset,input_dataset_val,label_dataset_val,type_number, fold=0):
        #print("*************************Training*************************")
        # 引数を追加して保存先を指定するよう改良
        # try_number:何回目の焼きなましかどうか。モデルの保存に使用

        # dataset training
        dataset_train2 = SegmentationDataSet0(
                                            #inputs=dataset_original,
                                            inputs=input_dataset,
                                            targets=label_dataset,
                                            transform=transforms_training)

        # dataloader training
        dataloader_training2 = DataLoader(dataset=dataset_train2,
                                        batch_size = BATCH_SIZE,
                                        shuffle=True)
        #もとはシャッフルtrue


        batch = next(iter(dataloader_training2))

        x, y = batch
        # print("x.shape = ", x.shape)
        # print("x.min(), x.max() = ", x.min(), x.max())
        # print("y.shape = ", y.shape)
        # print("torch.unique(y) = ", torch.unique(y))


        
        # dataset training
        dataset_val = SegmentationDataSet0(inputs=input_dataset_val,
                                            targets=label_dataset_val,
                                            transform=transforms_val)
        #書き換え箇所
        dataloader_val = DataLoader(dataset=dataset_val,
                                        batch_size = BATCH_SIZE,
                                        shuffle=False)
        
        
        
        ###earlystopping あり

        from unet import UNet
        from trainer2 import Trainer2 
        from torch import nn #import torch 
        from pytorchtools import EarlyStopping
        from torch.nn import BCEWithLogitsLoss
        from LuoTransUnet import getTransUnet

        #device
        if torch.cuda.is_available():
            device = torch.device('cuda') 
        else: 
            torch.device('cpu')
            #print("Before creating the UNet model: GPU was not available and CPU will be used instead")

        # custom logging the parameters of the UNet
        inChannels = IN_CHANNEL
        outChannels = 11
        #nBlocks = N_BLOCK
        startFilters = 32

        from customLog import custom_logger
        
        #custom_logger("/log/customLog.log", inChannels, outChannels, nBlocks, startFilters)

        #model
        # model = UNet(in_channels = inChannels,
        #             out_channels = outChannels,
        #             n_blocks = nBlocks, 
        #             start_filters=startFilters,
        #             activation='relu',
        #             normalization=NORMALIZATION,
        #             conv_mode='same',
        #             kernel_size=KERNEL_SIZE_CONV,
        #             kernel_out_channels=FUSION_OUT_CHANNEL,
        #             dim=2,
        #             ).to(device)
        model = getTransUnet.get_transNet(11, bMask=False, hidden_size=HIDDEN_SIZE, num_layers=N_LAYER, decoder_channels=DECODER_CHANNEL, patch_size=PATCH_SIZE, kernel_size=KERNEL_SIZE_CONV, kernel_out_channels=FUSION_OUT_CHANNEL)
        model = model.to(device=device)
        model_name = 'transunet'
        ### Later for experimenting
        #from originalUNet import Original_UNet
        #model = Original_UNet()

        # Assuming input_tensor is a sample input tensor with the correct shape (e.g., torch.randn(1, 3, 100, 100))
        input_tensor = torch.randn(1, 45, 100, 100).to(device)  # Adjust the shape as needed
        # print_model_shapes(model, input_tensor)


        #criterion
        criterion = torch.nn.CrossEntropyLoss()
        # criterion = BCEWithLogitsLoss()

        #optimizer
        optimizer = torch.optim.Adam(model.parameters(), lr=LR)

        #trainer
        trainer = Trainer2(model=model, 
                        device=device, 
                        criterion=criterion, 
                        optimizer=optimizer, 
                        training_DataLoader=dataloader_training2,
                        #validation_DataLoader=None, 
                        validation_DataLoader=dataloader_val, 
                        lr_scheduler=None, 
                        epochs=EPOCH, ##😺😺😺😺 epoch=0, 
                        notebook=True)

        #print("=======start training======")
        
        # start training
        training_losses, validation_losses, lr_rates = trainer.run_trainer()
        #print("***************************")

        
        
        #ここがちゃんとESで最適なエポック数のモデルになっているか要検証
        model_dir = os.path.join("model", date_str)
        if not os.path.exists(model_dir):
            os.makedirs(model_dir)
        model_name = f"model_fold_{fold}.pt"
        model_path = os.path.join(model_dir, model_name)
        torch.save(model.state_dict(), model_path)
        #print(f"modelname:{model_name}を保存しました")

        model_weights = torch.load(model_path)
        model.load_state_dict(model_weights)
        

        # images = annealing_input_dataset
        images = input_dataset_val

        from inference import predict
        from transformations import normalize_01, re_normalize
        # predict the segmentation maps 
        output = [predict(img, model, preprocess, postprocess, device) for img in images]


        # for i in range(len(input_name_val)):
        #     if(type_number == 0):
        #         cv2.imwrite(os.path.join(result_dir, f'{input_name_val[i]}.png'), output[i])
        #     elif(type_number == 1):
        #         cv2.imwrite(os.path.join(test_result_dir, 'result_original', f'{input_name_val[i]}.png'), output[i])
        #     elif(type_number == 2):
        #         cv2.imwrite(os.path.join(test_result_dir, 'result_test', f'{input_name_val[i]}.png'), output[i])


    import statistics
    # Dicecの計算
    def cal_DiceMulitple(dir, input_name):
        # change the label count as your preference
        Dice = [0] * 11
        Count1 = [0] * 11 #予測結果の各ラベルの要素数
        Count2 = [0] * 11 #ラベル画像の
        Count3 = [0] * 11 #正解した画素数
        
        for index in range(len(input_name)):
            #print('index = ', index)

            img1 = cv2.imread(dir + '/' + input_name[index] + ".png", cv2.IMREAD_GRAYSCALE)
            img2 = cv2.imread(labeled_dir + '/' + input_name[index] + ".png", cv2.IMREAD_GRAYSCALE)
            #_, img2 = cv2.threshold(img2, 0, 255, cv2.THRESH_BINARY)
            #print("予測画像:", dir  + '/' + input_name[index] + ".png")
            #print("テストラベル:", labeled_dir + '/' + input_name[index] + ".png")
            unique_label1 = np.unique(img1)
            unique_label2 = np.unique(img2)
            # print(unique_label1)
            # print(unique_label2)
            # change the image array size to your need
            for n in range(256):
                for l in range(256):
                    value1 = img1[n,l]
                    # for index, uq_value in enumerate(unique_label1):
                    #     if(value1 == uq_value):
                    #         value1 = index

                    Count1[value1] += 1

                    value2 = img2[n,l]
                    Count2[value2] += 1                    

                    if(value1 == value2):
                        Count3[value1] += 1 
        for i in range(11):
            if(Count1[i]+Count2[i] != 0):
                Dice[i] = (2*Count3[i])/(Count1[i] + Count2[i])
            if(Count1[i]+Count2[i] == 0):
                1
                #print("")
                #print("4 label case:" + str(input_name[index]))
        Dice.append(statistics.mean(Dice[1:]))
        #print('Count1 = ', Count1)
        #print('Count2 = ', Count2)
        #print('Count3 = ', Count3)
        #print('Dice = ', Dice)
        #print(unique_label)

        return Dice


    ## Test of 1x1conv
    import statistics
    from LuoTransUnet import getTransUnet
    # device
    def mConv_predict(test_input_name, fold=0):
        #print("*************************************Test*************************************")
        if torch.cuda.is_available():
            device = torch.device('cuda')
        else:
            #print("Using CPU instead of GPU")
            torch.device('cpu')

        # model = UNet(in_channels=IN_CHANNEL,  # check the parameters
        #             out_channels=11,
        #             n_blocks=N_BLOCK,
        #             start_filters=32,
        #             activation='relu',
        #             normalization=NORMALIZATION, #use instance when "batch" size is less than 10? batch
        #             # eric changed
        #             kernel_size=KERNEL_SIZE_CONV,
        #             kernel_out_channels=FUSION_OUT_CHANNEL,
        #             conv_mode='same',
        #             dim=2).to(device)
        model = getTransUnet.get_transNet(11, bMask=False, hidden_size=HIDDEN_SIZE, num_layers=N_LAYER, decoder_channels=DECODER_CHANNEL, patch_size=PATCH_SIZE, kernel_size=KERNEL_SIZE_CONV, kernel_out_channels=FUSION_OUT_CHANNEL)
        model = model.to(device=device)
        model_name = 'transunet'

        model_dir = os.path.join("model", date_str)
        #print("model_dir = ", model_dir)
        model_path = os.path.join(model_dir, f"model_fold_{fold}.pt") # load the model
        model_weights = torch.load(model_path)
        model.load_state_dict(model_weights)

        model.eval()

        # create test dataset
        test_input_dataset, _ = CreateWeightImage(test_input_name)


        # convert to torch
        test_input_tensor = torch.from_numpy(test_input_dataset).float().to(device)

        from inference import predict
        from transformations import normalize_01, re_normalize
        # predict
        output = [predict(img, model, preprocess, postprocess, device) for img in test_input_dataset]


        plot_output_img = list()
        
        
        fold_dir = os.path.join(test_result_dir, f"fold{fold}")
        os.makedirs(fold_dir, exist_ok=True)
        
        # save images
        for i, output_img in enumerate(output):
            #for checking the unique values of the result label
            labels = np.unique(output_img)

            output_image_path = os.path.join(fold_dir, f"{test_input_name[i]}.png")
            cv2.imwrite(output_image_path, output_img)
            
            # old writing when there was no stratifiedkfold
            # cv2.imwrite(os.path.join(test_result_dir, f"{test_input_name[i]}.png"), output_img)  # 根据需要调整文件路径和命名

            #print(labels)
            plot_output_img.append(output_img)


        Dice = cal_DiceMulitple(fold_dir, test_input_name)
        df = pd.DataFrame(Dice)
        df = df.T
        # HIDDEN_SIZE = trial.suggest_categorical('hidden_size', [512, 768, 1024, 1280])
        # N_LAYER = trial.suggest_categorical('n_layer', [8, 10, 12, 14, 16])
        # LR = trial.suggest_float('learning_rate', 1e-3, 1e-1)
        # KERNEL_SIZE_CONV = trial.suggest_categorical('kernel_size_conv', [1, 3, 5, 7, 9])
        # FUSION_OUT_CHANNEL = trial.suggest_categorical('post_fusion_out_channel', [1, 3, 5, 7, 9])
        # DECODER_CHANNEL = trial.suggest_categorical('decoder_channel', [(512, 256, 128, 64), (256, 128, 64, 32), (256, 128, 64, 16), (128, 64, 32, 16)])
        # PATCH_SIZE = trial.suggest_categorical('patch_size', [8, 16, 32])
        # BATCH_SIZE = trial.suggest_categorical('batch_size', [2, 4, 6, 8, 10, 12, 14, 16])
        # EPOCH = trial.suggest_int("epoch", 50, 300)
        param_string = f"Param values: (HIDDEN_SIZE: {HIDDEN_SIZE}, N_LAYER: {N_LAYER}, LR: {LR}, KERNEL_SIZE_CONV: {KERNEL_SIZE_CONV}, FUSION_OUT_CHANNEL: {FUSION_OUT_CHANNEL},  DECODER_CHANNEL:{DECODER_CHANNEL}, PATCH_SIZE:{PATCH_SIZE}, BATCH_SIZE:{BATCH_SIZE}, EPOCH:{EPOCH})"
        param_row = pd.DataFrame([[param_string] + [''] * (df.shape[1] - 1)])  # Adjust length of row to match columns
        df = pd.concat([param_row, df], ignore_index=True)
        df.to_csv(test_result_dir + "/Dice.csv", mode='a', header=False, index=False) # Diceの結果をcsvに追加
        return df.iloc[-1, -1]


    ## Test Learn_EA without annealing
    # training transformations and augmentations
    transforms_training = ComposeDouble([
        FunctionWrapperDouble(create_dense_target, input=False, target=True),
        FunctionWrapperDouble(np.moveaxis, input=True, target=False, source=-1, destination=0),
        FunctionWrapperDouble(normalize_01)
    ])

    #追加箇所ver3
    transforms_val =  ComposeDouble([
        FunctionWrapperDouble(create_dense_target, input=False, target=True),
        FunctionWrapperDouble(np.moveaxis, input=True, target=False, source=-1, destination=0),
        FunctionWrapperDouble(normalize_01)
    ])
    # random seed
    random_seed = 0

    X = file_names_with_prefix(data_dir, 'N')

    y_file_names = file_names_with_prefix(labeled_dir, 'N')
    y = [label_group[:2] for label_group in y_file_names]

    X = np.array(X)
    y = np.array(y)

    if CROSS_VAL:
        skf = StratifiedKFold(n_splits=N_SPLIT, shuffle=True, random_state=42)
        for fold, (train_index, test_index) in enumerate(skf.split(X, y), 1):
            # Split the data into train and test sets
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]
            
            # Split the training data further into train and validation ( 1/3 split)
            X_train_final, X_val, y_train_final, y_val = train_test_split(
                X_train, y_train, test_size=0.333333333, random_state=42, stratify=y_train)
            
            
            input_train = X_train_final
            input_name_val = X_val
            # print("Cross validation: " + str(CROSS_VAL))
            # print(f"Fold: {fold} out of {N_SPLIT}")
            # print("Augmentation: " + str(AUGMENTED))
            if AUGMENTED:
                1
                #print("Augmentation amount: " + str(AUGMENTATION))
            # print("Training: Total of " + str(len(input_train)) + " cases.")
            # print(input_train)

            # print("Validation: Total of " + str(len(input_name_val)) + " cases.")
            # print(input_name_val)
            
            # print("Test: Total of " + str(len(X_test)) + " cases.")  
            # print(X_test)
            # print()
            
            if AUGMENTED:
                ## This part is for accounting the data augmentation, attaches augmentation numbers after the original img
                ## ex: original number N1-1 -> N1-1-1, N1-1-2 etc.
                repeated_items_train = np.repeat(input_train, AUGMENTATION)
                suffixes_train = np.tile(np.arange(1, AUGMENTATION + 1), len(repeated_items_train))
                input_train = np.array([f"{item}-{suffix}" for item, suffix in zip(repeated_items_train, suffixes_train)])
                
                
                repeated_items_val = np.repeat(input_name_val, AUGMENTATION)
                suffixes_val = np.tile(np.arange(1, AUGMENTATION + 1), len(repeated_items_val))
                input_name_val = np.array([f"{item}-{suffix}" for item, suffix in zip(repeated_items_val, suffixes_val)])
            
            input_dataset,label_dataset = CreateWeightImage(input_train, AUGMENTED)
            input_dataset_val,label_dataset_val = CreateWeightImage(input_name_val, AUGMENTED)
            
            Learn_EA(input_dataset,label_dataset,input_dataset_val,label_dataset_val, 0, fold)
            mConv_predict(X_test, fold)
    else:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y, random_state=42)
        
        X_train_final, X_val, y_train_final, y_val = train_test_split(
                X_train, y_train, test_size=0.333333333, random_state=42, stratify=y_train)
        
        input_train = X_train_final
        input_name_val = X_val
        
        # print("Cross validation: " + str(CROSS_VAL))
        # print("Augmentation: " + str(AUGMENTED))
        if AUGMENTED:
            1
            #print("Augmentation amount: " + str(AUGMENTATION))
        # print("Training: Total of " + str(len(input_train)) + " cases.")
        # print(input_train)

        # print("Validation: Total of " + str(len(input_name_val)) + " cases.")
        # print(input_name_val)
            
        # print("Test: Total of " + str(len(X_test)) + " cases.")  
        # print(X_test)
        # print()
        
        if AUGMENTED:
            ## This part is for accounting the data augmentation, attaches augmentation numbers after the original img
            ## ex: original number N1-1 -> N1-1-1, N1-1-2 etc.
            repeated_items_train = np.repeat(input_train, AUGMENTATION)
            suffixes_train = np.tile(np.arange(1, AUGMENTATION + 1), len(repeated_items_train))
            input_train = np.array([f"{item}-{suffix}" for item, suffix in zip(repeated_items_train, suffixes_train)])
                
                
            repeated_items_val = np.repeat(input_name_val, AUGMENTATION)
            suffixes_val = np.tile(np.arange(1, AUGMENTATION + 1), len(repeated_items_val))
            input_name_val = np.array([f"{item}-{suffix}" for item, suffix in zip(repeated_items_val, suffixes_val)])
        
        input_dataset,label_dataset = CreateWeightImage(input_train, AUGMENTED)
        input_dataset_val,label_dataset_val = CreateWeightImage(input_name_val, AUGMENTED)
            
        Learn_EA(input_dataset,label_dataset,input_dataset_val,label_dataset_val, 0)
        last_dice = mConv_predict(X_test)
        torch.cuda.empty_cache()
        print("Last dice: " + str(last_dice))
        return last_dice
        
        
        

In [ ]:
import optuna

# Create a study to minimize or maximize the metric
study = optuna.create_study(direction='maximize')  # or 'minimize' based on your metric
study.optimize(objective, n_trials = 1000)

In [ ]:
print("Best trial:")
print(f"  Best Dice: {study.best_value}")  # Best objective value
print("  Params:")
for key, value in study.best_params.items():
    print(f"    {key}: {value}")


In [ ]:
df = study.trials_dataframe()
print(df.head())  # View the first few rows
df.to_csv("optuna_trials_n1000_mctransunet.csv", index=False)

In [ ]:
from optuna.visualization import plot_optimization_history
fig = plot_optimization_history(study)
fig.show()  # Display the plot


In [ ]:
from optuna.visualization import plot_param_importances
fig = plot_param_importances(study)
fig.show()


In [ ]:
from optuna.visualization import plot_parallel_coordinate
fig = plot_parallel_coordinate(study)
fig.show()


In [ ]:
from optuna.visualization import plot_contour
fig = plot_contour(study)
fig.show()


In [ ]:
from optuna.visualization import plot_slice
fig = plot_slice(study)
fig.show()

In [ ]:
from optuna.visualization import plot_edf
fig = plot_edf(study)
fig.show()

In [ ]:

import optuna
import pandas as pd
from datetime import datetime

# Load the trials DataFrame from the CSV
df = pd.read_csv("optuna_trials_n1000_mctransunet.csv")

top_trials = df.sort_values(by="value", ascending=False).head(10)

# Display the top 10 trials
print("Top 10 trials with the highest values:")
print(top_trials)


In [ ]:
trial_id = 733  # Replace with the desired trial ID

# Filter the DataFrame by the trial ID (assuming 'number' column corresponds to trial ID)
trial_info = df[df["number"] == trial_id]

# Check if the trial exists
if not trial_info.empty:
    print(f"Information for trial ID {trial_id}:")
    print(trial_info)
else:
    print(f"No trial found with ID {trial_id}.")